In [1]:
import os
import json
from collections import defaultdict

input_file = 'input_file.json'

In [2]:
tables = defaultdict(dict)

with open(input_file, 'r') as file:
    data = json.load(file)
    for key, value in data.items():
        tables[key] = value
print(tables.keys())

dict_keys(['2304.04370_1', '2304.04370_2', '2304.04370_3', '2308.12519_1', '2308.12519_2', '2308.12519_3', '2310.01444_2', '2310.01444_4', '2310.03965_1', '2310.03965_2', '2402.10890_1', '2402.10890_2', '2402.10890_3', '2403.04783_1', '2403.04783_2', '2403.04783_4', '2405.17129_1', '2405.17129_4', '2405.17129_5', '2406.03075_2', '2406.03075_3', '2406.03075_4', '2406.12707_1', '2406.12707_2', '2406.12707_3', '2407.16667_1', '2407.16667_3', '2407.16667_4', '2407.17115_2', '2407.17115_4', '2407.17115_5'])


In [3]:
import re
from bs4 import BeautifulSoup

def extract_claims_from_table(html):
    # Parsing del contenuto HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    # Trova la tabella
    table = soup.find('table')
    
    # Trova tutte le righe della tabella
    rows = table.find_all('tr')
    
    # Estrai il valore della cella in alto a sinistra (prima riga, prima colonna)
    table_specification_name = rows[0].find_all('td')[0].get_text(strip=True)
    
    # Estrai i nomi delle colonne dalla prima riga (escludendo la prima cella)
    headers = [header.get_text(strip=True) for header in rows[0].find_all('td')[1:]]
    
    claims = []
    claim_counter = 0
    
    # Itera sulle righe, a partire dalla seconda riga (escludi la riga di intestazione)
    for row in rows[1:]:
        cells = row.find_all('td')
        if len(cells) < 2:  # Salta le righe con meno di due celle
            continue
        
        # Estrai il valore della prima cella della riga corrente
        row_specification = cells[0].get_text(strip=True)
        
        # Crea una claim per ogni misura
        for i, value_cell in enumerate(cells[1:]):  # Escludi la prima cella
            measure_name = headers[i]  # Nome della misura dalla prima riga
            value = value_cell.get_text(strip=True)
            claims.append(f"claim {claim_counter}|{{{table_specification_name}, {row_specification}}} {measure_name}: {value}|")
            claim_counter += 1

    return claims

#|{Specification, Specification, …}, Measure, Outcome|

# HTML di esempio
table_html = tables['2405.17129_1']['table']  # Inserisci qui il tuo HTML completo

claims = extract_claims_from_table(table_html)

with open('claims_test.txt', 'a') as file:
    for claim in claims:
        file.write(claim + '\n')

# Stampa le claims
for claim in claims:
    print(claim)


claim 0|{Models, EXALT Baseline} F1-score: 0.43|
claim 1|{Models, EXALT Baseline} Precision: 0.43|
claim 2|{Models, EXALT Baseline} Recall: 0.44|
claim 3|{Models, ZSEC-gpt4turbo} F1-score: 0.55|
claim 4|{Models, ZSEC-gpt4turbo} Precision: 0.55|
claim 5|{Models, ZSEC-gpt4turbo} Recall: 0.58|
claim 6|{Models, ZSEC-gpt4o} F1-score: 0.57|
claim 7|{Models, ZSEC-gpt4o} Precision: 0.56|
claim 8|{Models, ZSEC-gpt4o} Recall: 0.60|
claim 9|{Models, MBCAWF} F1-score: 0.56|
claim 10|{Models, MBCAWF} Precision: 0.56|
claim 11|{Models, MBCAWF} Recall: 0.59|
claim 12|{Models, MIAWF-3444built on ZSEC-gpt4o and Ensemble-9} F1-score: 0.59|
claim 13|{Models, MIAWF-3444built on ZSEC-gpt4o and Ensemble-9} Precision: 0.59|
claim 14|{Models, MIAWF-3444built on ZSEC-gpt4o and Ensemble-9} Recall: 0.61|
claim 15|{Models, MIAWF-5555built on MIAWF-4 (which is built on MIAWF-3 and Ensemble-8) and Ensemble-8} F1-score: 0.60|
claim 16|{Models, MIAWF-5555built on MIAWF-4 (which is built on MIAWF-3 and Ensemble-8) and

In [4]:
from bs4 import BeautifulSoup
import pandas as pd

# Tabella HTML originale
html_table = tables['2402.10890_3']['table']

# Parsing HTML con BeautifulSoup
soup = BeautifulSoup(html_table, 'html.parser')

# Estrazione delle righe
rows = soup.find_all('tr')

# Intestazioni
headers = []
for cell in rows[0].find_all('td'):
    if 'colspan' in cell.attrs:
        for subheader in rows[1].find_all('td')[:int(cell['colspan'])]:
            headers.append(f"{cell.text.strip()} {subheader.text.strip()}")
    else:
        headers.append(cell.text.strip())

# Dati
data = []
for row in rows[2:]:
    data.append([cell.text.strip() for cell in row.find_all('td')])

# Creazione DataFrame
df = pd.DataFrame(data, columns=headers)

# Stampa il DataFrame
print(df)


                          Discriminators  \
0                          CodeLlama-13B   
1                          GPT-3.5-Turbo   
2                       CodeLlama-13B-FT   
3                         CodeLlama-13BE   
4                         GPT-3.5-TurboE   
5                      CodeLlama-13B-FTE   
6  Oracle Simulation (τ=1.0𝜏1.0\tau=1.0)   

  Spider (Greedy Gen = 62.3) Re-ranking  \
0                                  57.5   
1                                  58.3   
2                                  61.5   
3                                  65.5   
4                                  67.0   
5                                  70.3   
6                                  71.0   

  Spider (Greedy Gen = 62.3) Iter. Correct.  \
0                                      51.7   
1                                      52.7   
2                                      51.7   
3                                      62.0   
4                                      67.5   
5                   

In [5]:
import pandas as pd

def extract_claims_from_dataframe(df):
    claims = []
    claim_counter = 0
    
    # Assumiamo che la prima colonna contenga la Specification
    # e le colonne successive contengano le misure
    table_specification_name = df.columns[0]  # Il nome della prima colonna
    
    for idx, row in df.iterrows():
        row_specification = row[0]  # La prima colonna è la Specification per ogni riga
        
        # Per ogni misura nella riga, creiamo una claim
        for i, measure_value in enumerate(row[1:]):  # Escludiamo la prima colonna (Specification)
            measure_name = df.columns[i + 1]  # Il nome della misura (dalla colonna del DataFrame)
            value = measure_value
            claims.append(f"claim {claim_counter}|{{{table_specification_name}, {row_specification}}} {measure_name}: {value}|")
            claim_counter += 1
    
    return claims


# Estrai le claims dal DataFrame
claims = extract_claims_from_dataframe(df)

# Salva le claims in un file
with open('claims_test.txt', 'a') as file:
    for claim in claims:
        file.write(claim + '\n')

# Stampa le claims
for claim in claims:
    print(claim)



claim 0|{Discriminators, CodeLlama-13B} Spider (Greedy Gen = 62.3) Re-ranking: 57.5|
claim 1|{Discriminators, CodeLlama-13B} Spider (Greedy Gen = 62.3) Iter. Correct.: 51.7|
claim 2|{Discriminators, CodeLlama-13B} Spider (Greedy Gen = 62.3) Tree Search: 55.5|
claim 3|{Discriminators, CodeLlama-13B} Bird (Greedy Gen = 16.0) Re-ranking: 13.3|
claim 4|{Discriminators, CodeLlama-13B} Bird (Greedy Gen = 16.0) Iter. Correct.: 13.3|
claim 5|{Discriminators, CodeLlama-13B} Bird (Greedy Gen = 16.0) Tree Search: 13.3|
claim 6|{Discriminators, GPT-3.5-Turbo} Spider (Greedy Gen = 62.3) Re-ranking: 58.3|
claim 7|{Discriminators, GPT-3.5-Turbo} Spider (Greedy Gen = 62.3) Iter. Correct.: 52.7|
claim 8|{Discriminators, GPT-3.5-Turbo} Spider (Greedy Gen = 62.3) Tree Search: 56.2|
claim 9|{Discriminators, GPT-3.5-Turbo} Bird (Greedy Gen = 16.0) Re-ranking: 18.0|
claim 10|{Discriminators, GPT-3.5-Turbo} Bird (Greedy Gen = 16.0) Iter. Correct.: 17.3|
claim 11|{Discriminators, GPT-3.5-Turbo} Bird (Greedy G